# 모델 매개변수 최적화 하기
- 모델과 데이터가 준비되었으니, 데이터에 매개변수를 최적화하여 모델을 학습 검증 테스트를 할 것이다.
- 모델을 학습하는 과정은 반복적인 과정을 거치게 된다.
    - 각 단계에서 모델은 출력을 예측하고, 추측과 정답 사이 오류(손실(loss))를 계산하고, 매개 변수에 대한 오류의 도함수(derivate)를 수집한 뒤, 경사하강법을 통해 이 파라미터들을 최적화(optimize)한다.

In [2]:
# 기본 코드 (Pre-requisite code)
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(
    training_data,
    batch_size = 64
)
test_dataloader = DataLoader(
    test_data,
    batch_size = 64
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

100%|██████████| 26421880/26421880 [01:20<00:00, 329771.29it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 83242.48it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:25<00:00, 172286.17it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 369781.43it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



## 하이퍼파라미터 (Hyperparameter)
- 하이퍼 파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수이다.
- 서로 다른 하이퍼 파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있다.
- 학습 시 다음과 같은 기본적은 하이퍼 파라미터를 정의한다.
1. epoch 수 : 데이터셋을 반복하는 횟수
2. batch size : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
3. learning rate : 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작으면 학습 속도가 느리며, 값이 크면 학습 중 예측할 수 없는 동작을 범할 수 있다.

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 최적화 단계
- 하이퍼 파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다.
- 최적화 단계의 각 반복을 `epoch`이라고 부른다.
- 하나의 에폭은 다음 두 부분으로 구성된다.
1. 학습 단계(train loop) : 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴시킨다.
2. 검증/테스트 단계(validation/tset loop) : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복한다.
- 학습 단계에서 일어나는 몇가지 개념들을 살펴보자.

## 손실 함수(loss function)
- 학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 수 있다.
- **손실 함수(loss function)**은 획득한 결과와 실제 값 사이의 틀린 정도를 측정하며, 학습 중에 이 값을 최소화하려고 한다.
- 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)를 계산한다.
- 일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 `nn.MSELoss`(평균 제곱 오차)나, 분류(classification)에 사용하는 `nn.NLLLoss`(음의 로그 우도) 그리고 `nn.LogSoftmax`와 `nn.NLLLoss`를 합친 `nn.CrossEntropyLoss`등이 있다.
- 모델의 출력 로짓(logit)을 `nn.CrossEntropyLoss`에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산하자.

In [5]:
# 손실 함수를 초기화 한다.
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저(Optimizer)
- 최적화는 각 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정이다.
- **최적화 알고리즘**은 이 과정이 수행되는 방식(확률적 경사하강법, SGD)을 정의한다.
- 모든 최적화 절차(logic)은 `optimizer`객체에 캡슐화(encapsulate)된다.
- 여기서는 SGD 옵티마이저를 사용하고 있으며, Pythorch에는 ADAM이나 RMSProp같은 다른 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 존재한다.

- 학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록하여 옵티마이저를 초기화하자.

In [6]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr = learning_rate
)

- 학습 단계(loop)에서 최적화는 세단계로 이루어진다.
1. `optimizer.zero_grad()`를 호출하여 모델 매개변수의 변화도를 재설정한다. 기본적으로 변화도는 더해지기(add up) 때문에, 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정된다.
2. `loss.backward()`를 호출하여 예측 손실(prediction loss)를 역전파 한다. Pytorch는 각 매개변수에 대한 손실의 변화도를 저장한다.
3. 변화도를 계산한 뒤에는 `optimizer.step()`를 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정한다.

## 전체 구현
- 최적화 코드를 반복하여 수행하는 `train_loop`와 테스트 데이터로 모델의 성능을 측정하는 `test_loop`를 정의하였다.

In [8]:
def train_loop(
    datalodaer,
    model,
    loss_fn,
    optimizer
):
    size = len(datalodaer.dataset)
    for batch, (X,y) in enumerate(datalodaer):
        # 예측과 손실 계산
        pred = model(X)         # 입력에 대한 예측
        loss = loss_fn(pred, y) # loss 구하기
        
        # 역전파
        optimizer.zero_grad()   # 모델 매개변수의 변화도를 재설정. 반복마다 명시적 0초기화
        loss.backward()         # 예측 손실을 역전파 시킨다. 각 매개변수에 대한 손실의 변화도를 저장
        optimizer.step()        # 역전파 단계에서 수집된 변화도로 매개변수를 조정
        
        if batch % 100 == 0:
            loss = loss.item()
            current = (batch + 1 ) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
def test_loop(
    dataloader,
    model,
    loss_fn
):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
     

손실 함수와 옵티마이저를 초기화하고 `train_loop`와 `test_loop`에 전달한다.   
모델의 성능 향상을 알아보기 위해 자유롭게 에폭수를 증가할 수 있다.

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr = learning_rate
)

epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n------------------------")
    train_loop(
        datalodaer = train_dataloader,
        model = model,
        loss_fn = loss_fn,
        optimizer = optimizer
    )
    test_loop(
        dataloader = test_dataloader,
        model = model,
        loss_fn = loss_fn
    )

print("Done.")

Epoch 1
------------------------
loss: 2.306619  [   64/60000]
loss: 2.290363  [ 6464/60000]
loss: 2.268385  [12864/60000]
loss: 2.267946  [19264/60000]
loss: 2.249006  [25664/60000]
loss: 2.217978  [32064/60000]
loss: 2.223443  [38464/60000]
loss: 2.192082  [44864/60000]
loss: 2.198177  [51264/60000]
loss: 2.164495  [57664/60000]
Test Error: 
 Accuracy: 47.1%, Avg loss: 2.154685 

Epoch 2
------------------------
loss: 2.172257  [   64/60000]
loss: 2.157548  [ 6464/60000]
loss: 2.094066  [12864/60000]
loss: 2.118564  [19264/60000]
loss: 2.074581  [25664/60000]
loss: 2.003121  [32064/60000]
loss: 2.043066  [38464/60000]
loss: 1.956440  [44864/60000]
loss: 1.968955  [51264/60000]
loss: 1.915780  [57664/60000]
Test Error: 
 Accuracy: 55.6%, Avg loss: 1.895031 

Epoch 3
------------------------
loss: 1.931857  [   64/60000]
loss: 1.898123  [ 6464/60000]
loss: 1.773044  [12864/60000]
loss: 1.833382  [19264/60000]
loss: 1.729061  [25664/60000]
loss: 1.666484  [32064/60000]
loss: 1.708290  [

## Ref
[역전파 미적분학 3b1b](https://www.youtube.com/watch?v=tIeHLnjs5U8)   
[Loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions)   
[torch.optim](https://pytorch.org/docs/stable/optim.html)   
